In [1]:
from __future__ import print_function, division
#Import needed packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from onnx_tf.backend import prepare
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
#from pytorch2keras.converter import pytorch_to_keras
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics

plt.ion()   # interactive mode

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
# Mnist dataset fpr Lenet
batch_size = 128
T = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])
train_data = torchvision.datasets.MNIST('mnist_data', train=True, download=True, transform=T)
val_data = torchvision.datasets.MNIST('mnist_data', train=False, download=True, transform=T)

trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
valloader = torch.utils.data.DataLoader(val_data, batch_size = batch_size)

/opt/anaconda3/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
model_short_name = 'lenet5'
framework = 'pytorch'
ext = '.pth'
def validate(model, data):
    total = 0
    correct = 0
    since = time.time()
    for i, (images, labels) in enumerate(data):
        #images = images.cuda()
        x = model(images)
        value, pred = torch.max(x,1)
        pred = pred.data.cpu()
        total += x.size(0)
        correct += torch.sum(pred == labels)
    time_elapsed = time.time() - since
    #print('accuracy: {}%'.format(correct*100./total),'time {:.0f}m {:.0f}s, {}'.format(time_elapsed // 60, time_elapsed % 60, time_elapsed) )
    #return correct*100./total
    return correct*100./total, time_elapsed
def run_prediction():
    #for model in models: 
    data_accuracy = []
    data_time = []
    path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}'.format(framework,model_short_name)
    if os.path.exists(path):
        list_files = [x for x in os.listdir(path) if ext in x]
        for i in range(1, 11):
            for model_name in list_files:
                if '_{}{}'.format(i, ext) in model_name:
                    model = torch.load(path+'/'+model_name, map_location=torch.device('cpu'))
                    accuracy, time_ = validate(model, valloader)
                    data_accuracy.append(accuracy)
                    data_time.append(time_)
    print('Avg accuracy: {}, avg time: {}'.format(np.mean(data_accuracy), np.mean(data_time)))
run_prediction()

Avg accuracy: 98.98100280761719, avg time: 3.792650508880615


In [5]:
training_id = 10
model_short_name = 'lenet5'
framework = 'pytorch'

In [6]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/pytorch/lenet5/'

# Model class must be defined somewhere
since_0 = time.time()
model_name = 'torch_lenet5-mnist_2021-11-01_{}'.format(training_id)
model = torch.load(path+model_name+'.pth', map_location=torch.device('cpu'))
#resnet50_model.eval()
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+model_name+'.pth')
size0

252295

In [7]:
import pandas as pd 
path_output = '/Volumes/Cisco/Summer2022/onnx-exchange/analysis/prediction/'

#data_file = open(path_output + 'prediction_{}_{}.csv'.format(framework, model_name), mode='w', newline='',
#                                  encoding='utf-8')
#data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#data_writer.writerow(['model','framework', 'training_id', 'model_full', 'accuracy', 'precision', 'recall', 'f1'])


In [8]:
import numpy as np
def validate(model, data):
    total = 0
    correct = 0
    list_time = []
    for i, (images, labels) in enumerate(data):
        #images = images.cuda()
        since = time.time()
        x = model(images)
        value, pred = torch.max(x,1)
        pred = pred.data.cpu()
        total += x.size(0)
        correct += torch.sum(pred == labels)
        time_elapsed = time.time() - since
        list_time.append(time_elapsed)
        print('time {:.0f}m {:.0f}s, {}'.format(time_elapsed // 60, time_elapsed % 60, time_elapsed))
    #print('accuracy: {}%'.format(correct*100./total),'time {:.0f}m {:.0f}s, {}'.format(time_elapsed // 60, time_elapsed % 60, time_elapsed) )
    print('accuracy: {}%'.format(correct*100./total), np.mean(list_time) )
    return correct*100./total

In [9]:
validate(model, valloader)

time 0m 0s, 0.017175912857055664
time 0m 0s, 0.027454853057861328
time 0m 0s, 0.018689870834350586
time 0m 0s, 0.019691944122314453
time 0m 0s, 0.02837204933166504
time 0m 0s, 0.017062902450561523
time 0m 0s, 0.01658487319946289
time 0m 0s, 0.018651962280273438
time 0m 0s, 0.017510175704956055
time 0m 0s, 0.04961514472961426
time 0m 0s, 0.04036688804626465
time 0m 0s, 0.015864133834838867
time 0m 0s, 0.01955389976501465
time 0m 0s, 0.017236948013305664
time 0m 0s, 0.04600882530212402
time 0m 0s, 0.02584075927734375
time 0m 0s, 0.01802992820739746
time 0m 0s, 0.014750957489013672
time 0m 0s, 0.0161740779876709
time 0m 0s, 0.015019893646240234
time 0m 0s, 0.0262298583984375
time 0m 0s, 0.016291141510009766
time 0m 0s, 0.03246593475341797
time 0m 0s, 0.018787860870361328
time 0m 0s, 0.020048856735229492
time 0m 0s, 0.015372753143310547
time 0m 0s, 0.016318798065185547
time 0m 0s, 0.06452608108520508
time 0m 0s, 0.028105974197387695
time 0m 0s, 0.015468120574951172
time 0m 0s, 0.0337862968

tensor(99.0800)